In [179]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import silhouette_score

# Load raw peptide abundance data
df = pd.read_csv("filtered_data.csv")

# Identify drug concentration columns
start_col = 'aew541_1000nm'
start_idx = df.columns.get_loc(start_col)
drug_columns = df.columns[start_idx:]
filtered_drug_columns = [col for col in drug_columns if not col.endswith('_pdpd')]
dmso_columns = [col for col in filtered_drug_columns if col.endswith('_dmso')]
drug_names = sorted(set(col.split('_')[0] for col in filtered_drug_columns))

# Step 1: Log-transform all abundance values
for col in filtered_drug_columns:
    df[col] = np.log(df[col])

# Step 2: Compute DMSO log-median and log-std
df['Log_Median_Abundance'] = df[dmso_columns].median(axis=1, skipna=True)
df['Log_StdDev_Abundance'] = df[dmso_columns].std(axis=1, skipna=True)

# Step 3: Compute z-score of each drug concentration
for col in filtered_drug_columns:
    df[col] = np.where(
        np.isfinite(df['Log_StdDev_Abundance']) & (df['Log_StdDev_Abundance'] > 0),
        (df[col] - df['Log_Median_Abundance']) / df['Log_StdDev_Abundance'],
        np.nan
    )

# Output folders
os.makedirs("drug_clustermaps", exist_ok=True)
os.makedirs("drug_cluster_csvs", exist_ok=True)

# Step 4: Cluster peptides per drug
for drug in drug_names:  # remove [:2] to run on all drugs
    cols = [col for col in filtered_drug_columns if col.startswith(drug)]
    if len(cols) < 3:
        continue

    df_subset = df[df[cols].notna().sum(axis=1) >= 7].copy()
    df_subset = df_subset[np.isfinite(df_subset[cols]).all(axis=1)]
    if df_subset.empty or len(df_subset) < 5:
        print(f"⚠️ Skipped {drug} — not enough valid peptides.")
        continue

    try:
        linkage_matrix = linkage(df_subset[cols], method='average', metric='correlation')

        # Find best number of clusters using silhouette score
        best_k = None
        best_score = -1
        best_labels = None

        for k in range(2, min(9, len(df_subset))):
            labels = fcluster(linkage_matrix, t=k, criterion='maxclust')
            try:
                score = silhouette_score(df_subset[cols], labels, metric='correlation')
                if score > best_score:
                    best_k = k
                    best_score = score
                    best_labels = labels
            except Exception:
                continue

        if best_labels is None:
            print(f"❌ Could not compute silhouette score for {drug}")
            continue

        df_subset['Cluster'] = best_labels

        # Save heatmap
        cluster = sns.clustermap(
            df_subset[cols],
            method='average',
            metric='correlation',
            cmap='vlag',
            figsize=(10, 8)
        )
        cluster.fig.suptitle(f"{drug.upper()} Clustering (k={best_k}, score={best_score:.2f})", y=1.02)
        plt.savefig(f"drug_clustermaps/{drug}_clustermap.png", bbox_inches='tight')
        plt.close()

        # Save data with clusters
        export_cols = ['Variant'] + ['Variant ID'] + ['Proteins'] + cols + ['Cluster']
        df_subset[export_cols].to_csv(f"drug_cluster_csvs/{drug}_peptide_clusters.csv", index=False)
        print(f"✅ {drug}: {len(df_subset)} peptides clustered into {best_k} groups (score={best_score:.2f})")

    except Exception as e:
        print(f"❌ Error clustering {drug}: {e}")


/var/folders/q9/9xm1_d2s70s_3vxtd8hlfv3m0000gn/T/ipykernel_92283/3587265590.py:10: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("filtered_data.csv")
/var/folders/q9/9xm1_d2s70s_3vxtd8hlfv3m0000gn/T/ipykernel_92283/3587265590.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Log_Median_Abundance'] = df[dmso_columns].median(axis=1, skipna=True)
/var/folders/q9/9xm1_d2s70s_3vxtd8hlfv3m0000gn/T/ipykernel_92283/3587265590.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfr

✅ abemaciclib: 1124 peptides clustered into 2 groups (score=0.33)


/opt/anaconda3/lib/python3.12/site-packages/seaborn/matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)
/opt/anaconda3/lib/python3.12/site-packages/seaborn/matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)


✅ aew541: 1613 peptides clustered into 2 groups (score=0.65)


/opt/anaconda3/lib/python3.12/site-packages/seaborn/matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)
/opt/anaconda3/lib/python3.12/site-packages/seaborn/matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)


...

✅ bms754807: 1337 peptides clustered into 2 groups (score=0.34)


/opt/anaconda3/lib/python3.12/site-packages/seaborn/matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)
/opt/anaconda3/lib/python3.12/site-packages/seaborn/matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)


✅ bms777607: 1145 peptides clustered into 2 groups (score=0.41)
✅ bms911543: 837 peptides clustered into 2 groups (score=0.90)


/opt/anaconda3/lib/python3.12/site-packages/seaborn/matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)
/opt/anaconda3/lib/python3.12/site-packages/seaborn/matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)


✅ byl719: 1661 peptides clustered into 2 groups (score=0.57)


In [177]:
df.head()

,rowid,ccms_row_id,Variant,Variant ID,Unmod variant,Total,Total- Unmodified sequence,Variants- Unmodified sequence,Proteins,Mass,...,baricitib_10nm,baricitib_30000nm,baricitib_3000nm,baricitib_300nm,baricitib_30nm,baricitib_3nm,baricitib_dmso,baricitib_pdpd,Log_Median_Abundance,Log_StdDev_Abundance
0,5,5,.ESESTAGSFSLSVR.,21292,.ESESTAGSFSLSVR.,7995,8328,7,sp|P06239-2|LCK_HUMAN;sp|P06239-3|LCK_HUMAN;sp...,1456.7,...,1.445455,1.082329,1.158297,1.504993,0.876452,1.023389,1.033800,8137600.0,15.481801,0.856921
1,7,7,.IQDKEGIPPDQQR.,39596,.IQDKEGIPPDQQR.,6836,6882,7,sp|P0CG47|UBB_HUMAN;sp|P0CG48|UBC_HUMAN;sp|P62...,1523.8,...,1.372086,-0.734915,-0.501744,-0.033770,0.537989,0.604715,0.962047,5830600.0,15.399186,0.905393
2,11,11,.IFTSIGEDYDER.,36599,.IFTSIGEDYDER.,5284,5412,7,sp|P35232-2|PHB_HUMAN;sp|P35232|PHB_HUMAN;tr|C...,1444.6,...,2.142637,1.195260,0.994538,1.345024,1.867325,1.679094,1.633423,70372000.0,16.354748,1.088943
3,14,14,.TAVC+57.021DIPPR.,87369,.TAVCDIPPR.,4837,4837,1,sp|A6NNZ2|TBB8B_HUMAN;sp|P04350|TBB4A_HUMAN;sp...,1085.5,...,0.906103,-0.643357,-0.557229,0.259802,0.684262,1.074972,0.149373,3113400.0,14.575469,1.200130
4,15,15,.IITHPNFNGNTLDNDIMLIK.,37659,.IITHPNFNGNTLDNDIMLIK.,4830,20735,81,TRYP_PIG,2283.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.778331,1.679313


In [ ]:
import pandas as pd
import numpy as np

In [79]:
df = pd.read_csv('filtered_data.csv')
start_col = 'aew541_1000nm'
start_idx = df.columns.get_loc(start_col)
drug_columns = df.columns[start_idx:]
filtered_drug_columns = [col for col in drug_columns if not col.endswith('_pdpd')]

/var/folders/q9/9xm1_d2s70s_3vxtd8hlfv3m0000gn/T/ipykernel_92283/1317422812.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('filtered_data.csv')


In [81]:
# Parse protein lists and explode
df['Protein_List'] = df['Proteins'].str.split(';')
df_exploded = df.explode('Protein_List')
df_exploded['Protein'] = df_exploded['Protein_List'].str.strip().str.split('|').str[-1]
df_exploded.head()

,rowid,ccms_row_id,Variant,Variant ID,Unmod variant,Total,Total- Unmodified sequence,Variants- Unmodified sequence,Proteins,Mass,...,baricitib_10nm,baricitib_30000nm,baricitib_3000nm,baricitib_300nm,baricitib_30nm,baricitib_3nm,baricitib_dmso,baricitib_pdpd,Protein_List,Protein
0,5,5,.ESESTAGSFSLSVR.,21292,.ESESTAGSFSLSVR.,7995,8328,7,sp|P06239-2|LCK_HUMAN;sp|P06239-3|LCK_HUMAN;sp...,1456.7,...,18264000.0,13380000.0,14280000.0,19220000.0,11216000.0,12721000.0,12835000.0,8137600.0,sp|P06239-2|LCK_HUMAN,LCK_HUMAN
0,5,5,.ESESTAGSFSLSVR.,21292,.ESESTAGSFSLSVR.,7995,8328,7,sp|P06239-2|LCK_HUMAN;sp|P06239-3|LCK_HUMAN;sp...,1456.7,...,18264000.0,13380000.0,14280000.0,19220000.0,11216000.0,12721000.0,12835000.0,8137600.0,sp|P06239-3|LCK_HUMAN,LCK_HUMAN
0,5,5,.ESESTAGSFSLSVR.,21292,.ESESTAGSFSLSVR.,7995,8328,7,sp|P06239-2|LCK_HUMAN;sp|P06239-3|LCK_HUMAN;sp...,1456.7,...,18264000.0,13380000.0,14280000.0,19220000.0,11216000.0,12721000.0,12835000.0,8137600.0,sp|P06239|LCK_HUMAN,LCK_HUMAN
0,5,5,.ESESTAGSFSLSVR.,21292,.ESESTAGSFSLSVR.,7995,8328,7,sp|P06239-2|LCK_HUMAN;sp|P06239-3|LCK_HUMAN;sp...,1456.7,...,18264000.0,13380000.0,14280000.0,19220000.0,11216000.0,12721000.0,12835000.0,8137600.0,tr|E9PAP0|E9PAP0_HUMAN,E9PAP0_HUMAN
0,5,5,.ESESTAGSFSLSVR.,21292,.ESESTAGSFSLSVR.,7995,8328,7,sp|P06239-2|LCK_HUMAN;sp|P06239-3|LCK_HUMAN;sp...,1456.7,...,18264000.0,13380000.0,14280000.0,19220000.0,11216000.0,12721000.0,12835000.0,8137600.0,tr|E9PJ92|E9PJ92_HUMAN,E9PJ92_HUMAN


In [89]:
# Group by protein and calculate median across peptides
protein_abundance = df_exploded.groupby('Protein')[filtered_drug_columns].median().reset_index()
protein_abundance.head()

,Protein,aew541_1000nm,aew541_100nm,aew541_10nm,aew541_30000nm,aew541_3000nm,aew541_300nm,aew541_30nm,aew541_3nm,aew541_dmso,...,barasertibhqpa_dmso,baricitib_1000nm,baricitib_100nm,baricitib_10nm,baricitib_30000nm,baricitib_3000nm,baricitib_300nm,baricitib_30nm,baricitib_3nm,baricitib_dmso
0,1433B_HUMAN,3587700.0,7000700.0,5116200.0,2788900.0,3514250.0,7838000.0,3679200.0,4916000.0,3782300.0,...,2513300.0,5105900.0,13988400.0,18874500.0,4644400.0,6073900.0,10381000.0,13510000.0,7630600.0,12022150.0
1,1433E_HUMAN,6590750.0,9298300.0,8225800.0,6299800.0,4053200.0,8037700.0,3794600.0,10863000.0,6647500.0,...,4165800.0,4303100.0,13386000.0,17947000.0,6163750.0,5490000.0,5861100.0,12812000.0,7482150.0,11721500.0
2,1433F_HUMAN,3865350.0,6869150.0,4436400.0,2009980.0,5323675.0,6349610.0,2038605.0,4497950.0,4803100.0,...,5465450.0,3163300.0,15132100.0,20634950.0,4242100.0,4009650.0,6221000.0,13730400.0,9627450.0,16413150.0
3,1433G_HUMAN,5043800.0,9940000.0,9571800.0,4861800.0,7088900.0,8931600.0,4699600.0,9715200.0,12187000.0,...,3462750.0,5164600.0,9335900.0,14570700.0,7138700.0,8466000.0,13131000.0,12653000.0,8595000.0,7797350.0
4,1433S_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1007400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
# Step 4: Compute per-protein statistics
protein_abundance['Mean_Abundance'] = protein_abundance[filtered_drug_columns].mean(axis=1, skipna=True)
protein_abundance['Median_Abundance'] = protein_abundance[filtered_drug_columns].median(axis=1, skipna=True)
protein_abundance['StdDev_Abundance'] = protein_abundance[filtered_drug_columns].std(axis=1, skipna=True)

protein_abundance.head()

,Protein,aew541_1000nm,aew541_100nm,aew541_10nm,aew541_30000nm,aew541_3000nm,aew541_300nm,aew541_30nm,aew541_3nm,aew541_dmso,...,baricitib_10nm,baricitib_30000nm,baricitib_3000nm,baricitib_300nm,baricitib_30nm,baricitib_3nm,baricitib_dmso,Mean_Abundance,Median_Abundance,StdDev_Abundance
0,1433B_HUMAN,3587700.0,7000700.0,5116200.0,2788900.0,3514250.0,7838000.0,3679200.0,4916000.0,3782300.0,...,18874500.0,4644400.0,6073900.0,10381000.0,13510000.0,7630600.0,12022150.0,5.631055e+06,3115550.0,7.630601e+06
1,1433E_HUMAN,6590750.0,9298300.0,8225800.0,6299800.0,4053200.0,8037700.0,3794600.0,10863000.0,6647500.0,...,17947000.0,6163750.0,5490000.0,5861100.0,12812000.0,7482150.0,11721500.0,6.284524e+06,3937475.0,1.204857e+07
2,1433F_HUMAN,3865350.0,6869150.0,4436400.0,2009980.0,5323675.0,6349610.0,2038605.0,4497950.0,4803100.0,...,20634950.0,4242100.0,4009650.0,6221000.0,13730400.0,9627450.0,16413150.0,4.470106e+06,2098100.0,1.070048e+07
3,1433G_HUMAN,5043800.0,9940000.0,9571800.0,4861800.0,7088900.0,8931600.0,4699600.0,9715200.0,12187000.0,...,14570700.0,7138700.0,8466000.0,13131000.0,12653000.0,8595000.0,7797350.0,6.773007e+06,4469825.0,8.049686e+06
4,1433S_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.662346e+06,2271500.0,9.272627e+06


In [93]:
# Normalize by protein median and log transform
for col in filtered_drug_columns:
    protein_abundance[col] = np.where(
        protein_abundance['Median_Abundance'] > 0,
        np.log(protein_abundance[col] / protein_abundance['Median_Abundance']),
        np.nan
    )
protein_abundance.head()

,Protein,aew541_1000nm,aew541_100nm,aew541_10nm,aew541_30000nm,aew541_3000nm,aew541_300nm,aew541_30nm,aew541_3nm,aew541_dmso,...,baricitib_10nm,baricitib_30000nm,baricitib_3000nm,baricitib_300nm,baricitib_30nm,baricitib_3nm,baricitib_dmso,Mean_Abundance,Median_Abundance,StdDev_Abundance
0,1433B_HUMAN,0.141106,0.809604,0.496006,-0.110758,0.120420,0.922578,0.166290,0.456089,0.193927,...,1.801406,0.399256,0.667595,1.203572,1.467024,0.895761,1.350345,5.631055e+06,3115550.0,7.630601e+06
1,1433E_HUMAN,0.515127,0.859292,0.736736,0.469978,0.028967,0.713603,-0.036961,1.014823,0.523701,...,1.516883,0.448146,0.332389,0.397798,1.179843,0.641981,1.090885,6.284524e+06,3937475.0,1.204857e+07
2,1433F_HUMAN,0.611020,1.186008,0.748811,-0.042907,0.931132,1.107361,-0.028766,0.762590,0.828229,...,2.285954,0.704026,0.647672,1.086898,1.878580,1.523586,2.057051,4.470106e+06,2098100.0,1.070048e+07
3,1433G_HUMAN,0.120811,0.799218,0.761472,0.084059,0.461181,0.692246,0.050128,0.776342,1.003021,...,1.181663,0.468181,0.638709,1.077627,1.040545,0.653831,0.556435,6.773007e+06,4469825.0,8.049686e+06
4,1433S_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.662346e+06,2271500.0,9.272627e+06


In [95]:
# protein_abundance.to_csv('protein_level_log_ratio_abundance.csv', index=False)